In [1]:
# Libraries
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch.nn.functional as f
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

In [3]:
# Load Iris dataset from Sklearn
iris = load_iris()
x = iris.data
y = iris.target


In [4]:
# Split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

x_train=torch.FloatTensor(x_train)
x_test=torch.FloatTensor(x_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)